# Asana Library

In [14]:
#| default_exp asana.asana

In [15]:
#| exporti

from __future__ import annotations

from dotenv import load_dotenv
import os
from dataclasses import dataclass, field
from typing import List
import datetime as dt
from dateutil.parser import parse as dtu_parse
from mdutils.mdutils import MdUtils

from nbdev.showdoc import patch_to

import domolibrary.client.get_data as gd

In [16]:
#|export

@dataclass
class AsanaAuth:
    token: str
    workspace_id : str
    base_url = 'https://app.asana.com/api/1.0'

    auth_header : dict = field(repr = False, default = None)

    def __post_init__(self):
        self.generate_auth_header()

    def get_auth_token(self):
        self.generate_auth_header()
        return self.token


    def generate_auth_header(self):
        self.auth_header = {
            'authorization': f'Bearer {self.token}',
        }

        return self.auth_header

In [17]:
#| exporti
def get_date(datefield):
    return dtu_parse(datefield) if datefield else None

In [18]:
#| export
@dataclass
class AsanaUser:
    id: str
    name: str
    resource_type: str

    @classmethod
    def _from_json(cls, obj):
        return cls(
            id=obj['gid'],
            name=obj['name'],
            resource_type=obj['resource_type']
        )
    
@dataclass
class AsanaProject:
    id: str
    name: str
    workspace_id:str
    permalink_url: str
    auth : AsanaAuth = field(repr = False)

    is_archived: bool
    is_completed: bool

    created_date: dt.datetime
    modified_date: dt.datetime
        
    owner: AsanaUser
    members: List[AsanaUser]
    
    due_date: dt.datetime=None
    completed_date: dt.datetime = None

    tasks : List[AsanaTask] = None

    @classmethod
    def _from_json(cls, obj, auth = None):

        owner = AsanaUser._from_json(obj.get('owner')) if obj.get('owner') else None

        members = [AsanaUser._from_json(member) for member in obj.get('members', [])] if obj.get('members') else None
        
        created_date = get_date(obj.get('created_at'))
        modified_date = get_date(obj.get('modified_at')) 
        due_date = get_date(obj.get('due_on'))
        completed_date = get_date(obj.get('completed_at'))

        return cls(
            auth = auth,
            id=obj.get('gid'),
            name=obj.get('name'),
            workspace_id=obj.get('workspace', {}).get('gid'),
            permalink_url=obj.get('permalink_url'),
            is_archived=obj.get('archived'),
            is_completed=obj.get('completed'),
            created_date=created_date,
            modified_date=modified_date,
            owner=owner,
            members=members,
            due_date=due_date,
            completed_date=completed_date
        )


In [19]:
#| export
@dataclass
class AsanaSection:
    id: str
    name: str
    resource_type: str

    @classmethod
    def _from_json(cls, data: dict) -> 'AsanaSection':
        return cls(
            id=data.get('gid'),
            name=data.get('name'),
            resource_type=data.get('resource_type')
        )

@dataclass
class AsanaMembership:
    project: AsanaProject
    section: AsanaSection

    @classmethod
    def _from_json(cls, data: dict) -> 'AsanaMembership':
        project = AsanaProject._from_json(data['project']) if data.get('project') else None
        section = AsanaSection._from_json(data['section']) if data.get('section') else None
        return cls(
            project=project,
            section=section
        )
    
    def to_text(self):
        return f'{self.project.name} -> {self.section.name}' 

In [20]:
#|exporti
@patch_to(AsanaProject, cls_method=True)
async def get_projects(cls : AsanaProject, auth : AsanaAuth, debug_api: bool = False, 
    debug_num_stacks_to_drop=1, return_raw : bool = False
    ):

    url = f'{auth.base_url}/projects'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             )
    if return_raw:
        return res

    return [AsanaProject._from_json(proj_obj, auth = auth) for proj_obj in res.response['data']]


In [28]:
load_dotenv('../.env')

asana_client_id = os.environ['ASANA_PUBLIC_CLIENT']
asana_client_secret = os.environ['ASANA_PUBLIC_CLIENT_SECRET']
asana_pat = os.environ['ASANA_PUBLIC_PAT']

asana_workspace_id = '9169352463943'
auth = AsanaAuth(token = asana_pat, workspace_id = asana_workspace_id)

asana_projects = await AsanaProject.get_projects(auth, debug_api= False)
asana_projects


[AsanaProject(id='1206043185525448', name='asana_sync', workspace_id=None, permalink_url=None, is_archived=None, is_completed=None, created_date=None, modified_date=None, owner=None, members=None, due_date=None, completed_date=None, tasks=None)]

In [22]:
#| exporti
@patch_to(AsanaProject, cls_method=True)
async def get_by_id(cls: AsanaProject,auth : AsanaAuth, project_id, debug_api: bool = False, 
debug_num_stacks_to_drop=1, return_raw: bool = False
):
    url = f'{auth.base_url}/projects/{project_id}'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             )
    
    if return_raw: 
        return res

    return cls._from_json(res.response['data'], auth = auth)

    

In [30]:
asana_project = await AsanaProject.get_by_id(auth = auth, project_id = asana_projects[0].id)


In [25]:
#| export
@dataclass
class AsanaTask:
    id: str
    name: str
    workspace_id : str

    auth : AsanaAuth = field(repr= False)
    assignee: AsanaUser = None
    
    assignee_status: str = None

    is_completed: bool = None
    
    created_at: dt.DateTime = None
    completed_on: dt.DateTime = None
    due_on: dt.DateTime = None
    modified_at: dt.DateTime = None

    memberships: List[dict] = None

    notes: str = None

    parent: Optional[dict] = None
    permalink_url: str = None
    projects: List[AsanaProject] = None
    stories : List[AsanaStory] = None

    tags: List[dict] = None

    @classmethod
    def _from_json(cls, obj: dict, auth: AsanaAuth) -> AsanaTask:
        
        assignee = AsanaUser._from_json(obj.get('assignee')) if obj.get('assignee') else None
        
        projects = [AsanaProject._from_json(project) for project in obj.get('projects', [])]
        
        memberships = [AsanaMembership._from_json(member_obj) for member_obj in obj.get('memberships', []) ]
        return cls(
            id=obj.get('gid'),
            name=obj.get('name'),
            auth=auth,
            workspace_id=obj.get('workspace', {}).get('gid') ,
            
            assignee=assignee,

            is_completed=obj.get('completed'),
            
            assignee_status=obj.get('assignee_status'),
            
            completed_on=get_date(obj.get('completed_at')),
            created_at=get_date(obj.get('created_at')),
            due_on = get_date(obj.get('due_on')),
            modified_at=get_date(obj.get('modified_at')),
            
            memberships=memberships,
            notes=obj.get('notes'),
            parent=obj.get('parent'),
            permalink_url=obj.get('permalink_url'),
            tags=obj.get('tags', []),

            
            projects=projects,
        )

In [40]:
#| exporti
@patch_to(AsanaProject)
async def get_tasks(self, debug_api: bool = False, 
    debug_num_stacks_to_drop=1, return_raw :bool = False
):
    auth = self.auth
    params = {'project' : self.id}

    url = f'{auth.base_url}/tasks'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             params=params
                             )
    
    if return_raw:
        return res

    return [AsanaTask._from_json(task_obj, auth = auth) for task_obj in res.response['data']]

    



In [42]:
asana_tasks = await asana_project.get_tasks( debug_api= False, return_raw = False)
asana_tasks


[AsanaTask(id='1206043185525463', name='Draft project brief', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], stories=None, tags=[]),
 AsanaTask(id='1206043185525465', name='Schedule kickoff meeting', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], stories=None, tags=[]),
 AsanaTask(id='1206043185525467', name='Share timeline with teammates', workspace_id=None, assignee=None, assignee_status=None, is_completed=None, created_at=None, completed_on=None, due_on=None, modified_at=None, memberships=[], notes=None, parent=None, permalink_url=None, projects=[], stories=None, tags=[])]

In [43]:
#| exporti
@patch_to(AsanaTask, cls_method=True)
async def get_by_id(cls, auth, task_id, 
    debug_api: bool = False, debug_num_stacks_to_drop=1, return_raw :bool = False
):

    url = f'{auth.base_url}/tasks/{task_id}'

    res = await gd.get_data(auth=auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             )
    
    if return_raw:
        return res

    return cls._from_json(res.response['data'], auth = auth)



In [52]:
asana_task = await AsanaTask.get_by_id(task_id=asana_tasks[0].id, 
    auth= auth, return_raw= False)

from pprint import pprint
pprint(asana_task)
# # asana_task.response['data']



AsanaTask(id='1206043185525463',
          name='Draft project brief',
          workspace_id='1206043147237389',
          assignee=AsanaUser(id='1206043147237379',
                             name='Jae Wilson',
                             resource_type='user'),
          assignee_status='inbox',
          is_completed=False,
          created_at=datetime.datetime(2023, 11, 27, 22, 34, 14, 216000, tzinfo=tzutc()),
          completed_on=None,
          due_on=datetime.datetime(2023, 11, 29, 0, 0),
          modified_at=datetime.datetime(2023, 11, 27, 22, 46, 52, 900000, tzinfo=tzutc()),
          memberships=[AsanaMembership(project=AsanaProject(id='1206043185525448',
                                                            name='asana_sync',
                                                            workspace_id=None,
                                                            permalink_url=None,
                                                            is_archived=None,
    

In [48]:
#| export
@dataclass
class AsanaStory:
    id: str
    created_at: dt.DateTime
    created_by: AsanaUser
    text: str
    type: str
    resource_subtype: str

    auth : AsanaAuth  = field(repr=False)

    @classmethod
    def _from_json(cls, data: dict, auth : AsanaAuth) -> AsanaStory:
        created_by = AsanaUser._from_json(data['created_by']) if data.get('created_by') else None

        return cls(
            id=data.get('gid'),
            created_at = get_date(data.get('created_at')) ,
            created_by=created_by,
            text=data.get('text'),
            type=data.get('type'),
            resource_subtype=data.get('resource_subtype'),
            auth = auth
        )

    def to_text(self):
        return f'{self.created_at.date()} - {self.created_by.name} - {self.text}'
        

In [57]:
#| exporti
@patch_to(AsanaTask)
async def get_stories(self, 
    debug_api: bool = False, debug_num_stacks_to_drop=1, return_raw :bool = False, 
    is_only_comments: bool = False
):

    url = f'{auth.base_url}/tasks/{self.id}/stories'

    res = await gd.get_data(auth=self.auth, method='GET',
                             url=url,
                             num_stacks_to_drop=debug_num_stacks_to_drop,
                             debug_api=debug_api,
                             )
    
    if return_raw:
        return res
    
    self.stories =  [AsanaStory._from_json(story_obj, auth = auth) for story_obj in res.response['data']]

    if is_only_comments:
        self.stories = [story for story in self.stories if 'comment' in story.resource_subtype ]
    
    return self.stories


In [60]:

await asana_task.get_stories(return_raw=False, is_only_comments=True, debug_api= False)


[AsanaStory(id='1206043165446912', created_at=datetime.datetime(2023, 11, 27, 22, 49, 29, 310000, tzinfo=tzutc()), created_by=AsanaUser(id='1206043147237379', name='Jae Wilson', resource_type='user'), text='we need to get this done fast!', type='comment', resource_subtype='comment_added')]

In [61]:
#| exporti
@patch_to(AsanaTask)
def to_md(self, output_folder = 'markdown'):

    print(self.__dict__)

    if not os.path.exists(output_folder):
        os.mkdir(output_folder)

    def treat_field(key, value, mdFile):
        print(value)
        if isinstance(value, AsanaUser):
            mdFile.new_line(f'{key} - {value.name}')
            return

        if isinstance(value, dt.datetime):
            mdFile.new_line(f'{key} - {value.date()}')
            return
        
        if key == 'memberships':
            mdFile.new_header(level=1, title=key)
            [mdFile.new_line(asana_class.to_text()) for asana_class in value]

            return
        
        if key == 'stories':
            mdFile.new_header(level=1, title=key)
            [mdFile.new_line(asana_class.to_text()) for asana_class in value if 'comment' in asana_class.resource_subtype]

            return


        if key in ['name', 'projects'] or not value:
            return
            
        
        return mdFile.new_line(f'{key} - {value}')

        
    mdFile = MdUtils(file_name=f'{output_folder}/{self.id}', title=self.name)
    [ treat_field(key, getattr( self,key), mdFile) for key in self.__dict__.keys() if key not in ['auth','workspace_id']] 
    mdFile.create_md_file()
    # mdFile.new_table(columns=3, rows=6, text=list_of_strings, text_align='center')


In [62]:
await asana_task.get_stories()

asana_task.to_md('sample')

{'id': '1206043185525463', 'name': 'Draft project brief', 'workspace_id': '1206043147237389', 'auth': AsanaAuth(token='2/1206043147237379/1206043196710290:ffdb99ecfd96835a9555e33d5422ca8d', workspace_id='9169352463943'), 'assignee': AsanaUser(id='1206043147237379', name='Jae Wilson', resource_type='user'), 'assignee_status': 'inbox', 'is_completed': False, 'created_at': datetime.datetime(2023, 11, 27, 22, 34, 14, 216000, tzinfo=tzutc()), 'completed_on': None, 'due_on': datetime.datetime(2023, 11, 29, 0, 0), 'modified_at': datetime.datetime(2023, 11, 27, 22, 46, 52, 900000, tzinfo=tzutc()), 'memberships': [AsanaMembership(project=AsanaProject(id='1206043185525448', name='asana_sync', workspace_id=None, permalink_url=None, is_archived=None, is_completed=None, created_date=None, modified_date=None, owner=None, members=None, due_date=None, completed_date=None, tasks=None), section=AsanaSection(id='1206043185525449', name='To do', resource_type='section'))], 'notes': 'Do the thing mr. wilso